# Combining Multiple Excel Files with Pandas

A common data wrangling task is to aggregate data from multiple spreadsheets and files.

This notebook will walk through the basic flow required to parse multiple excel files, combine some data, clean it up and analyze it.

## The Problem

Here is a diagram showing the challenges with the common process used in businesses all over the world to consolidate data from multiple Excel files, clean up and perform some analysis.

<img src="Excel-File-Processing.png">

The typical approach involves cutting and pasting data or writing painful VBA code.

Python + pandas can be a great alternative that is much more scaleable and powerful.

<img src="Excel-File-Processing-pandas.png">

## Collecting the Data

In [ ]:
import pandas as pd
import numpy as np

Let's take a look at the files in our input directory, using the convenient shell commands in python.

In [ ]:
!ls

We only want to look at the sales .xlsx files.

In [ ]:
!ls sales*.xlsx

Use the python glob module to easily list out the files we need

In [ ]:
import glob

In [ ]:
glob.glob("sales*.xlsx")

Let's import each of our files and combine them into one file. Panda's `concat` and `append` can do this for us. We'll use append in this example.

The code snippet below will initialize a blank DataFrame then append all of the individual files into the `all_data` DataFrame.

In [ ]:
all_data = pd.DataFrame()
for f in glob.glob("sales*.xlsx"):
    df = pd.read_excel(f)
    all_data = all_data.append(df,ignore_index=True)

Now we have all the data in our `all_data` DataFrame.

In [ ]:
all_data.describe()

It looks like a complete dataset with no missing values.

In [ ]:
all_data.head()

It is not critical in this example but the best practice is to convert the date column to a date time object.

In [ ]:
all_data['date'] = pd.to_datetime(all_data['date'])
all_data.info()

## Combining Data

The next thing we want to do is read in another file that contains the customer status by account. You can think of this as a company's customer segmentation strategy or some other mechanism for identifying their customers.

First, we read in the data.

In [ ]:
status = pd.read_excel("customer-status.xlsx")
status

We want to merge this data with our concatenated data set of sales. We use panda's `merge` function and tell it to do a left join which is similar to Excel's vlookup function.

In [ ]:
all_data_st = pd.merge(all_data, status, how='left')
all_data_st.head()

Let's look at a specific account.

In [ ]:
all_data_st[all_data_st["account number"]==737550].head()

This account number was not in our status file, so we have a bunch of NaN's. We can decide how we want to handle this situation. For this specific case, let's label all missing accounts as bronze. Use the `fillna`.

In [ ]:
all_data_st['status'].fillna('bronze',inplace=True)
all_data_st.head()

Check the data just to make sure we're all good.

In [ ]:
all_data_st[all_data_st["account number"]==737550].head()

Now we have all of the data along with the status column filled in. We can do our normal data manipulations using the full suite of pandas capability.

## Using Categories

One of the relatively new functions in pandas is support for categorical data. From the pandas, documentation

>"Categoricals are a pandas data type, which correspond to categorical variables in statistics: a variable, which can take on only a limited, and usually fixed, number of possible values (categories; levels in R). Examples are gender, social class, blood types, country affiliations, observation time or ratings via Likert scales."

For our purposes, the status field is a good candidate for a category type.

First, we typecast it to a category using `astype`.

In [ ]:
all_data_st["status"] = all_data_st["status"].astype("category")

This doesn't immediately appear to change anything yet.

In [ ]:
all_data_st.head()

Buy you can see that it is a new data type.

In [ ]:
all_data_st.dtypes

Categories get more interesting when you assign order to the categories. Right now, if we call sort on the column, it will sort alphabetically. 

In [ ]:
all_data_st.sort_values(by=["status"]).head()

We use set_categories to tell it the order we want to use for this category object. In this case, we use the Olympic medal ordering.

In [ ]:
 all_data_st["status"].cat.set_categories([ "gold","silver","bronze"],inplace=True)

Now, we can sort it so that gold shows on top.

In [ ]:
all_data_st.sort_values(by=["status"]).head()

In [ ]:
all_data_st["status"].describe()

For instance, if you want to take a quick look at how your top tier customers are performing compared to the bottom. Use `groupby` to give us the average of the values.

In [ ]:
all_data_st.groupby(["status"])["quantity","unit price","ext price"].mean()

Of course, you can run multiple aggregation functions on the data to get really useful information 

In [ ]:
all_data_st.groupby(["status"])["quantity","unit price","ext price"].agg([np.sum,np.mean, np.std])

So, what does this tell you? One observation is that more units are sold to our bronze customers than gold. Even when you look at the total dollar value associated with bronze vs. gold, it looks backwards.

Maybe we should look at how many bronze customers there are and see what is going on.

First, we'll filter out the unique accounts and see how many gold, silver and bronze customers there are.

In [ ]:
all_data_st.drop_duplicates(subset=["account number","name"]).iloc[:,[0,1,7]].groupby(["status"])["name"].count()

Ok. This makes a little more sense. We see that there are nine bronze customers and only four gold customers. That is probably why the volumes are so skewed towards the bronze customers.